This is where we can build the pipeline running application. We can add and refer to python files that define classes and so on. Let's see how it goes.

In [1]:
import random
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import torch.cuda

# inspect torch path file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("deam_processed/best_model.pth", map_location=device)
print(checkpoint.keys())

# Since we already have embeddings we can skip instantiating the model here.

#get VGG
# vggish = hub.load('https://tfhub.dev/google/vggish/1')
# vggish

odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'output.weight', 'output.bias'])


#### Let's create a regression step (embeddings -> V/A)

In [2]:
from torch import nn
import numpy as np

x_test = np.load("deam_processed/splits/X_test.npy")
y_test = np.load("deam_processed/splits/y_test.npy")

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")

torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

CUDA available: False
CUDA version: None
GPU count: 0
Using cpu device


### Data Comparison

In [79]:
import pandas as pd

# grab MERGE data
# VGGish
verbose = True
lm_predictions = np.load("data/Audio Features and Model/extracted_features/extracted_features/results/linear_model_predictions.npz")
if verbose:
    print('lm_pred')
    print(lm_predictions.files)
    print(lm_predictions["train_pred"].shape)
    print(lm_predictions["train_pred"])

nn_predictions = np.load("data/Audio Features and Model/extracted_features/extracted_features/results/nn_predictions.npz")
if verbose:
    print('nn_pred')
    print(nn_predictions.files)
    print(nn_predictions["train_pred"].shape)
    print(nn_predictions["train_pred"])

#X_audio
ids_train = np.load("data/Audio Features and Model/extracted_features/extracted_features/splits/ids_train.npy")
if verbose:
    print('ids_train')
    print(ids_train.shape)
    print(ids_train)

ids_val = np.load("data/Audio Features and Model/extracted_features/extracted_features/splits/ids_val.npy")
print('ids_val')
print(ids_val.shape)
if verbose:
    print(ids_val)

ids_test = np.load("data/Audio Features and Model/extracted_features/extracted_features/splits/ids_test.npy")
print('ids_test')
print(ids_test.shape)
if verbose:
    print(ids_test)

# RoBERTa
# X_lyrics
panda_merge_lyrics_pred = pd.read_csv("data/Lyrics Features and Model/MERGE_Lyrics_Complete/merge_lyrics_complete_av_values.csv")
if verbose:
    print(panda_merge_lyrics_pred.columns)
    print(panda_merge_lyrics_pred.head())

# Y
Y_pred = pd.read_csv("data/MERGE_Bimodal_Complete/merge_bimodal_complete_av_values.csv")
if verbose:
    print(Y_pred.columns)
    print(Y_pred.head())


lm_pred
['train_pred', 'train_true', 'val_pred', 'val_true', 'test_pred', 'test_true']
(1551, 2)
[[0.43982995 0.33138043]
 [0.30436885 0.44202358]
 [0.26988444 0.22830135]
 ...
 [0.38348094 0.22942466]
 [0.33037326 0.26724023]
 [0.50084233 0.3577829 ]]
nn_pred
['train_pred', 'train_true', 'val_pred', 'val_true', 'test_pred', 'test_true']
(1551, 2)
[[0.45719957 0.40246072]
 [0.35674843 0.5339292 ]
 [0.45476738 0.3570533 ]
 ...
 [0.49934745 0.3789434 ]
 [0.456428   0.39545193]
 [0.62258387 0.5089669 ]]
ids_train
(1551,)
['MT0008709705' 'MT0006800841' 'A188' ... 'MT0028293340' 'MT0012658386'
 'MT0005540907']
ids_val
(332,)
['MT0003320924' 'MT0005660001' 'A190' 'MT0033470806' 'MT0028617797'
 'MT0005001966' 'MT0001276044' 'MT0004131260' 'MT0008537015'
 'MT0011818239' 'MT0010169019' 'MT0007190016' 'MT0005115042'
 'MT0032643023' 'MT0003748227' 'MT0034403159' 'MT0002485672'
 'MT0004759588' 'MT0001638394' 'MT0010054305' 'MT0013280170'
 'MT0002915967' 'MT0000808227' 'MT0031661393' 'MT0028593700'

In [71]:
%load_ext autoreload
%autoreload 2

from FAI_LinRegClassifier import FAI_LinRegClassifier
from FAI_Sklearner import FAI_Sklearner

epochs = 100
verbose2 = False
def extract_features(audio_ids, audio_pred, lyrics_pred, truth_pred):
    """
    Extract V/A features from three data sets representing the audio, lyrics and true predictions of VA values.
    :param audio_ids: file of ids to identity songs in split
    :param audio_pred: A/V prediction of audio features
    :param lyrics_pred: A/V prediction of lyrics features
    :param truth_pred: A/V true features
    :return: np.arrays X, Y, shape N,4 and N,2 respectively
    """
    X = np.empty((audio_ids.shape[0], 4))
    Y = np.empty((audio_ids.shape[0], 2))
    count = 0
    for i in range(audio_ids.shape[0]):
        song_id = audio_ids[i] # get song id in training splits
        Y_row = truth_pred[truth_pred["Audio_Song"] == song_id]
        lyrics_id = Y_row["Lyric_Song"]
        if len(lyrics_id) == 0:
            print("song_id: ", song_id, ", missing in lyrics_id")
            continue
        if verbose2:
            print(lyrics_id)
        lyrics_id = lyrics_id.item()
        lyricsVA = np.array(lyrics_pred[lyrics_pred["Song"] == lyrics_id].values[0][1:3])
        lyricsVA = np.flip(lyricsVA)
        audioVA = np.flip(np.array(audio_pred[i])) # CONFIRM IF FLIP REQUIRED
        y_bimodal = np.flip(Y_row.values[0][2:4])
        xEntry = np.concatenate([lyricsVA, audioVA])
        X[i] = xEntry
        Y[i] = y_bimodal
        count+=1
    print('run stats')
    print('count: ', count)
    print('X shape: ', X.shape)
    print('Y shape: ', Y.shape)
    return X, Y

# nn_predications: ['train_pred', 'train_true', 'val_pred', 'val_true', 'test_pred', 'test_true']
X, Y = extract_features(ids_train, nn_predictions['train_pred'], panda_merge_lyrics_pred, Y_pred)

classifier = FAI_LinRegClassifier(iterations=epochs)
if True:
    print('X-shape: ', X.shape)
    print('Y-shape: ',Y.shape)

# classifier.train(X, Y)
N = int(1e4)
X_test = np.zeros((N, 2))
for i in range(N):
    inc = (1/N)*i
    X_test[i:] = np.array((inc, 1.0-inc))
y_test = np.zeros(N)
y_test[:] = 1
# print('\ntest run:')
# for i in range(1):
#     print('run [',i,']')
#     testSkClassifier = FAI_Sklearner(iterations=int(10))
#     testSkClassifier.train(X_valence=X_test, X_arousal=X_test, Y_valence=y_test, Y_arousal=y_test)
#     for i in range(N):
#         a = np.random.rand(N)
#         b = 1-a
#         X_v = np.column_stack((a, b))
#     testValResult = testSkClassifier.validate(X_valence=X_v, X_arousal=X_v, Y_valence=y_test, Y_arousal=y_test)
# print(testValResult)
#
print('\nrun')
if False:
    print(X)
    print('v: ', X[:, [0, 2]])
    print('a: ', X[:, [1, 3]])
print(nn_predictions)
X_val, Y_val = extract_features(ids_val, nn_predictions['val_pred'], panda_merge_lyrics_pred, Y_pred)
# print(np.subtract(X_val, X))
# print('val ', ids_val.shape)
# print('train', ids_train.shape)
# print(np.intersect1d(ids_train, ids_val))
# print('X_val: ', X_val.shape)
# print('X: ', X.shape)
# print('val_pred: ', nn_predictions['val_pred'].shape)
# print('val_train: ', nn_predictions['train_pred'].shape)
# print(np.sum(np.subtract(X_val, X)))

for i in range(1, 12, 2):
    print('i:', i)
    skClassifier = FAI_Sklearner(iterations=int(5*i))
    skClassifier.train(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
    t_result, tv_loss, ta_loss = skClassifier.validate(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
    result, vv_loss, va_loss = skClassifier.validate(X_valence=X_val[:, [0, 2]], X_arousal=X_val[:, [1, 3]], Y_valence=Y_val[:, 0], Y_arousal=Y_val[:, 1])
    # Calculate overfitting metric
    vgap = vv_loss - tv_loss
    vgap_ratio = vv_loss / tv_loss
    agap = va_loss - ta_loss
    agap_ratio = va_loss / ta_loss
    print(f"Gap: valance {vgap:.6f} | arousal {agap:.6f}")
    print(f"Ratio: valence {vgap_ratio:.2f} | arousal {agap_ratio:.2f}")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
run stats
count:  1551
X shape:  (1551, 4)
Y shape:  (1551, 2)
X-shape:  (1551, 4)
Y-shape:  (1551, 2)

run
NpzFile 'data/Audio Features and Model/extracted_features/extracted_features/results/nn_predictions.npz' with keys: train_pred, train_true, val_pred, val_true, test_pred...
run stats
count:  332
X shape:  (332, 4)
Y shape:  (332, 2)
i: 1
final weights:  [[0.49251331 0.09920158]
 [0.29868227 0.1260768 ]] biases: [[0.22073925]
 [0.27850552]]
total losses:  0.013929988555372906 0.009616307492956007
total losses:  0.013840233843646797 0.011387316327648311
Gap: valance -0.000090 | arousal 0.001771
Ratio: valence 0.99 | arousal 1.18
i: 3
final weights:  [[0.65873129 0.06359255]
 [0.33083227 0.11310713]] biases: [[0.15050075]
 [0.26943385]]
total losses:  0.00634477564440234 0.008741156890844117
total losses:  0.006249911582862888 0.010358837270789482
Gap: valance -0.000095 | arousal 0.001618
Ratio: 

C:\Users\khaba\music_mood_classification\.venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1579: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\khaba\music_mood_classification\.venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1579: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [73]:
alphas = [0.0001, 0.001, 0.01, 0.10, 1]
# 0.0001 (default) best
for a in alphas:
    print('alpha=', a)
    skClassifier = FAI_Sklearner(alpha=a, iterations=10)
    skClassifier.train(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
    t_result, tv_loss, ta_loss = skClassifier.validate(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
    result, vv_loss, va_loss = skClassifier.validate(X_valence=X_val[:, [0, 2]], X_arousal=X_val[:, [1, 3]], Y_valence=Y_val[:, 0], Y_arousal=Y_val[:, 1])
    # Calculate overfitting metric
    vgap = vv_loss - tv_loss
    vgap_ratio = vv_loss / tv_loss
    agap = va_loss - ta_loss
    agap_ratio = va_loss / ta_loss
    print(f"Gap: valance {vgap:.6f} | arousal {agap:.6f}")
    print(f"Ratio: valence {vgap_ratio:.2f} | arousal {agap_ratio:.2f}")

alpha= 0.0001
final weights:  [[0.63833352 0.06785201]
 [0.32937241 0.10931597]] biases: [[0.15761647]
 [0.27162772]]
total losses:  0.0070927886871537645 0.008765457268528815
total losses:  0.006994848961347588 0.010383836960033897
Gap: valance -0.000098 | arousal 0.001618
Ratio: valence 0.99 | arousal 1.18
alpha= 0.001
final weights:  [[0.63196375 0.06858429]
 [0.33103945 0.1095257 ]] biases: [[0.1615148]
 [0.2714652]]
total losses:  0.007354210588366176 0.008727734684111152
total losses:  0.007256092509326894 0.010341362612667729
Gap: valance -0.000098 | arousal 0.001614
Ratio: valence 0.99 | arousal 1.18
alpha= 0.01
final weights:  [[0.59467143 0.06132497]
 [0.31137762 0.10318768]] biases: [[0.18112407]
 [0.28523141]]
total losses:  0.008853502649799831 0.009225303811374302
total losses:  0.008736907248192071 0.010917640113472713
Gap: valance -0.000117 | arousal 0.001692
Ratio: valence 0.99 | arousal 1.18
alpha= 0.1
final weights:  [[0.32961286 0.03983634]
 [0.17281487 0.04881898]]

C:\Users\khaba\music_mood_classification\.venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1579: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\khaba\music_mood_classification\.venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1579: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\khaba\music_mood_classification\.venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1579: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [77]:
models = ['Sgdregressor', 'Linear', 'Ridge', 'Lasso', 'ElasticNet']
# 0.0001 (default) best
for m in models:
    print('model=', m)
    skClassifier = FAI_Sklearner( iterations=10, model=m)
    skClassifier.train(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
    t_result, tv_loss, ta_loss = skClassifier.validate(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
    result, vv_loss, va_loss = skClassifier.validate(X_valence=X_val[:, [0, 2]], X_arousal=X_val[:, [1, 3]], Y_valence=Y_val[:, 0], Y_arousal=Y_val[:, 1])
    # Calculate overfitting metric
    vgap = vv_loss - tv_loss
    vgap_ratio = vv_loss / tv_loss
    agap = va_loss - ta_loss
    agap_ratio = va_loss / ta_loss
    print(f"Gap: valance {vgap:.6f} | arousal {agap:.6f}")
    print(f"Ratio: valence {vgap_ratio:.2f} | arousal {agap_ratio:.2f}")

model= Sgdregressor
final weights:  [[0.63621622 0.0713809 ]
 [0.33607659 0.11092921]] biases: [[0.15911165]
 [0.26978237]]
total losses:  0.007207011630419757 0.008615419859869276
total losses:  0.00711294161283182 0.010213427419571392
Gap: valance -0.000094 | arousal 0.001598
Ratio: valence 0.99 | arousal 1.19
model= Linear
final weights:  [[ 0.99736746 -0.00407842]
 [ 0.99427389 -0.00501187]] biases: [0.00400886 0.00594789]
total losses:  0.0001326558415495013 0.0001525280698763127
total losses:  0.0001625611285203357 0.00021751406817604443
Gap: valance 0.000030 | arousal 0.000065
Ratio: valence 1.23 | arousal 1.43
model= Ridge
final weights:  [[ 0.99736626 -0.00407826]
 [ 0.99427056 -0.00501165]] biases: [0.00400939 0.00594938]
total losses:  0.0001326558416266952 0.0001525280700900342
total losses:  0.00016256038989800057 0.00021751504623950896
Gap: valance 0.000030 | arousal 0.000065
Ratio: valence 1.23 | arousal 1.43
model= Lasso
final weights:  [[ 0.99545491 -0.        ]
 [ 0.9

C:\Users\khaba\music_mood_classification\.venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1579: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [80]:
cross_skClassifier = FAI_Sklearner( iterations=10, model=m)
cross_skClassifier.train(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])
t_result, tv_loss, ta_loss = cross_skClassifier.validate(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])

result, vv_loss, va_loss = cross_skClassifier.validate(X_valence=X_val[:, [0, 2]], X_arousal=X_val[:, [1, 3]], Y_valence=Y_val[:, 0], Y_arousal=Y_val[:, 1])

cross_skClassifier.cross_validate(X_valence=X[:, [0, 2]], X_arousal=X[:, [1, 3]], Y_valence=Y[:, 0], Y_arousal=Y[:, 1])

final weights:  [[ 0.99545914 -0.        ]
 [ 0.98894614 -0.00200372]] biases: [0.00304002 0.00701192]
total losses:  0.00013295257750777363 0.00015320860972763972
total losses:  0.00016136178945397789 0.00021933731649420745
Valence CV MSE: 0.000133 (+/- 0.000045)
Arousal CV MSE: 0.000155 (+/- 0.000063)


In [81]:
X_test, Y_test = extract_features(ids_test, nn_predictions['test_pred'], panda_merge_lyrics_pred, Y_pred)
res, vloss, aloss = cross_skClassifier.validate(X_valence=X_test[:, [0, 2]], X_arousal=X_test[:, [1, 3]], Y_valence=Y_test[:, 0], Y_arousal=Y_test[:, 1])

print(f"\n{'='*60}")
print("FINAL TEST SET PERFORMANCE")
print(f"{'='*60}")
print(f"Test MSE: valence={vloss:.6f}, arousal={aloss:.6f}")

run stats
count:  333
X shape:  (333, 4)
Y shape:  (333, 2)
total losses:  5.73250994029328e-05 5.875311196558517e-05

FINAL TEST SET PERFORMANCE
Test MSE: valence=0.000057, arousal=0.000059
